In [1]:
import cv2 
import numpy as np
import dlib
import math
from scipy.spatial import Delaunay                  
from scipy.interpolate import RectBivariateSpline 
from PIL import Image, ImageDraw

In [2]:
class Triangle:
    def __init__(self, vertices):
        if isinstance(vertices, np.ndarray) == 0:
            raise ValueError("Input argument is not of type np.array.")
        if vertices.shape != (3, 2):
            raise ValueError("Input argument does not have the expected dimensions.")
        if vertices.dtype != np.float64:
            raise ValueError("Input argument is not of type float64.")
        self.vertices = vertices

    # Credit to https://github.com/zhifeichen097/Image-Morphing 
    def getPoints(self):
        width = round(max(self.vertices[:, 0]) + 2)
        height = round(max(self.vertices[:, 1]) + 2)
        mask = Image.new('P', (width, height), 0)
        ImageDraw.Draw(mask).polygon(tuple(map(tuple, self.vertices)), outline=255, fill=255)
        coordArray = np.transpose(np.nonzero(mask))
        return coordArray

class Morpher: 
    def __init__(self, sourceImagePath,targetImagePath):
        self.sourceImage = cv2.imread(sourceImagePath)
        self.targetImage = cv2.imread(targetImagePath)
        self.sourcePoints = self.getPointsOnFace(self.sourceImage)
        self.targetPoints = self.getPointsOnFace(self.targetImage)
        self.triangles = self.loadTriangles(self.sourcePoints,self.targetPoints)
        self.sourceTriangles = self.triangles[0]
        self.targetTriangles = self.triangles[1] 
        self.newSourceImage = np.zeros((self.sourceImage.shape[0]+1, self.sourceImage.shape[1]+1, 3), dtype=np.float64)
        self.newTargetImage = np.zeros((self.sourceImage.shape[0]+1, self.sourceImage.shape[1]+1, 3), dtype=np.float64)
        
        
    def getTriangle(self,points) :
        triangles = Delaunay(points)
        triangleList = []
        coordinatesTriangle = points[triangles.simplices]
        for triangle in coordinatesTriangle :
            triangleList.append(Triangle(triangle.astype(np.float64)))
        return triangleList
        
    def loadTriangles(self,leftPoints, rightPoints):
        leftTriangles = []
        rightTriangles = []
        leftSimplice = Delaunay(leftPoints).simplices
        rightSimplice = Delaunay(rightPoints).simplices
        for triangle in leftSimplice.tolist():
            Tri = Triangle(leftPoints[triangle].astype(np.float64))
            Tri1 = Triangle(rightPoints[triangle].astype(np.float64))
            leftTriangles.append(Tri) 
            rightTriangles.append(Tri1)  
        return tuple((leftTriangles, rightTriangles))
    
    def getPointsOnFace(self,image) :
        points = []
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect the faces
        rects = detector(gray)

        # go through the face bounding boxes
        for rect in rects:
            # extract the coordinates of the bounding box
            x1 = rect.left()
            y1 = rect.top()
            x2 = rect.right()
            y2 = rect.bottom()

            # apply the shape predictor to the face ROI
            shape = predictor(gray, rect)

            for n in range(0, 68):
                x = shape.part(n).x
                y = shape.part(n).y
                points.append([x, y])
            
            xlen = image.shape[0]
            ylen = image.shape[1]
            points.append([0,0])
            points.append([ylen/2,0])
            points.append([ylen,0])
            
            points.append([0,xlen/2])
            points.append([ylen,xlen/2])
            
            points.append([0,xlen])
            points.append([ylen/2,xlen])
            points.append([ylen,xlen])
            
            
        return np.array(points)
    
    def Transformation(self, originalImage, targetImage, Target, original,isleft):  # this is image
        NbyTwo = Target.getPoints()
        xp, yp = np.transpose(NbyTwo)
        x = self.calcXPoints(xp, yp)
        y = self.calcYPoints(xp, yp)
        xdim = np.arange(np.amin(original.vertices[:, 1]), np.amax(original.vertices[:, 1]), 1)
        ydim = np.arange(np.amin(original.vertices[:, 0]), np.amax(original.vertices[:, 0]), 1)
     
        xyVal = originalImage[int(xdim[0]):int(xdim[-1] + 1), int(ydim[0]):int(ydim[-1] + 1)]
        
        bilinearRed = RectBivariateSpline(xdim, ydim, xyVal[:,:,0], kx=1, ky=1)
        bilinearGreen = RectBivariateSpline(xdim, ydim, xyVal[:,:,1], kx=1, ky=1)
        bilinearBlue = RectBivariateSpline(xdim, ydim, xyVal[:,:,2], kx=1, ky=1)
        
        if isleft : 
            self.newSourceImage[xp, yp,0] = bilinearRed.ev(x,y)
            self.newSourceImage[xp, yp,1] = bilinearGreen.ev(x,y)
            self.newSourceImage[xp, yp,2] = bilinearBlue.ev(x,y)
        else : 
            self.newTargetImage[xp, yp,0] = bilinearRed.ev(x,y)
            self.newTargetImage[xp, yp,1] = bilinearGreen.ev(x,y)
            self.newTargetImage[xp, yp,2] = bilinearBlue.ev(x,y)
              
        
    def getImageAtAlpha(self,alpha):
        leftTarget = np.zeros(self.sourceImage.shape, dtype = 'float64') 
        rightTarget = np.zeros(self.targetImage.shape, dtype = 'float64')
        self.interpolatedTriangles = []
        array_len = len(self.sourceTriangles)
        for i in range(0, array_len):
            imageMidTargets = (1 - alpha) * self.sourceTriangles[i].vertices + alpha * self.targetTriangles[i].vertices 
            self.interpolatedTriangles.append(Triangle(imageMidTargets))
        for sourceTriangle, interpolatedTriangle in zip(self.sourceTriangles, self.interpolatedTriangles):
            self.affineMatrix(sourceTriangle, interpolatedTriangle)
            self.Transformation(self.sourceImage,leftTarget, interpolatedTriangle, sourceTriangle,0)
        for targetTriangle, interpolatedTriangle in zip(self.targetTriangles, self.interpolatedTriangles):
            self.affineMatrix(targetTriangle, interpolatedTriangle)
            self.Transformation(self.targetImage, rightTarget, interpolatedTriangle, targetTriangle,1)  
        final = (alpha * self.newSourceImage + (1-alpha) * self.newTargetImage).astype(np.uint8)
        return final
    
    def affineMatrix(self, original, interpolatedTriangle):
        self.interpolatedTriangle = interpolatedTriangle
        self.original = original
        A = np.array([[original.vertices[0, 0], original.vertices[0, 1], 1, 0, 0, 0],
                  [0, 0, 0, original.vertices[0, 0], original.vertices[0, 1], 1],
                  [original.vertices[1, 0], original.vertices[1, 1], 1, 0, 0, 0],
                  [0, 0, 0, original.vertices[1, 0], original.vertices[1, 1], 1],
                  [original.vertices[2, 0], original.vertices[2, 1], 1, 0, 0, 0],
                  [0, 0, 0, original.vertices[2, 0], original.vertices[2, 1], 1]])
        b = np.reshape(interpolatedTriangle.vertices, (6, 1))
        h = np.linalg.solve(A, b)
        matrix = np.vstack([np.reshape(h, (2, 3)), [0, 0, 1]])
        self.inverseMatrix = np.linalg.inv(matrix)
        return self.inverseMatrix
        
    def calcXPoints(self, x, y):
        return self.inverseMatrix[1, 1] * x + self.inverseMatrix[1, 0] * y + self.inverseMatrix[1, 2]

    def calcYPoints(self, x, y):
        return self.inverseMatrix[0, 1] * x + self.inverseMatrix[0, 0] * y + self.inverseMatrix[0, 2]
    

In [3]:
#CREATION D UNE SEQUENCE D IMAGES  
morph = Morpher("king.jpg","cena.jpg")

dossier = "temp/"
images = []
for i in range(0,5) : 
    img = morph.getImageAtAlpha(i/5)
    imgFormat = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    PIL_image = Image.fromarray(np.uint8(imgFormat)).convert('RGB')
    images.append(PIL_image)
    output_path = dossier + str(i) +".jpg"
    PIL_image.save(output_path)

In [17]:
#CREATION DE LA VIDEO 

morph = Morpher("paire3-1.jpg","paire2-2.jpg")


fps = 30
duration = 2
videodims = (morph.sourceImage.shape[1],morph.sourceImage.shape[0])

writer = cv2.VideoWriter("output.mp4",cv2.VideoWriter_fourcc(*"avc1"),fps,videodims)
for i in range(fps*duration):
    writer.write(morph.getImageAtAlpha(i/(fps*duration)))
writer.release()



In [14]:
# dessiner les points sur un visage
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

image = cv2.imread("hfi.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect the faces
rects = detector(gray)

# go through the face bounding boxes
for rect in rects:
    # extract the coordinates of the bounding box
    x1 = rect.left()
    y1 = rect.top()
    x2 = rect.right()
    y2 = rect.bottom()

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

    # apply the shape predictor to the face ROI
    shape = predictor(gray, rect)
    
    for n in range(0, 68):
        x = shape.part(n).x
        y = shape.part(n).y
        cv2.circle(image, (x, y), 4, (255, 0, 0), -1)
        
cv2.imwrite("hfi_point.jpg", image)

194

In [18]:
# dessiner les triangles sur un visage
from PIL import Image, ImageDraw

# Charger l'image

image = Image.open("ofin.jpg")
image2 = Image.open("hfi.jpg")

# Obtenir les dimensions de l'image source
width, height = image.size

# Créer une nouvelle image blanche avec les mêmes dimensions
white_image = Image.new('RGB', (width, height), 'white')

morph = Morpher("ofin.jpg","hfi.jpg")

# Créer un objet Draw pour manipuler l'image
draw = ImageDraw.Draw(image)
draw2 = ImageDraw.Draw(image2)
draw_white = ImageDraw.Draw(white_image)


lefttri = morph.triangles[0]
righttri = morph.triangles[1]


for left,right in zip(lefttri,righttri) :
    triangle_points = [(left.vertices[0][0], left.vertices[0][1]), (left.vertices[1][0], left.vertices[1][1]), (left.vertices[2][0], left.vertices[2][1])]
    draw.polygon(triangle_points, outline="blue")
    #draw.polygon(triangle_points, fill = (255,0,0))
    
    triangle_points2 = [(right.vertices[0][0], right.vertices[0][1]), (right.vertices[1][0], right.vertices[1][1]), (right.vertices[2][0], right.vertices[2][1])]
    draw2.polygon(triangle_points2, outline="blue")
    #draw2.polygon(triangle_points2, fill = (255,0,0))
    
    triangle_points3 = [(right.vertices[0][0]*0.5 + left.vertices[0][0] * 0.5 , right.vertices[0][1]*0.5 + left.vertices[0][1] * 0.5), (right.vertices[1][0]*0.5 + left.vertices[1][0] * 0.5, right.vertices[1][1]*0.5 + left.vertices[1][1] * 0.5), (right.vertices[2][0]*0.5 + left.vertices[2][0] * 0.5, right.vertices[2][1]*0.5 + left.vertices[2][1] * 0.5)]
    draw_white.polygon(triangle_points3, outline="blue")
    #draw2.polygon(triangle_points2, fill = (255,0,0))
    
    
    
image2.save("tri_hilary.jpg")
image.save("tri_obama.jpg")
white_image.save("tri_white.jpg")

# Afficher l'image (facultatif)
image.show()

image2.show()

white_image.show()

